In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import pi

from sklearn.linear_model import RidgeCV
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

from pathlib import Path
import datetime as dt
import os

from google.cloud import bigquery
from google.oauth2 import service_account

import numpy as np, pandas as pd
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
from statsmodels.stats.diagnostic import acorr_ljungbox

In [2]:
# Información del proyecto y autenticación a BQ
project_id = "enersinc-tbsg-bq"
key_path = "C:\BigQuery\eramirez-tbsg.json"

# Cargar las credenciales del archivo JSON
credentials = service_account.Credentials.from_service_account_file(key_path)

# Crear el cliente de BigQuery
client = bigquery.Client(project=project_id, credentials=credentials)

In [3]:
FechaIni='2024-01-01'
FechaFin='2025-11-30'

In [4]:
# Consulta a la maestra de recursos
query = rf"""
select * from `enersinc-tbsg-bq`.tbsg.public_ddem
where fechaoperacion>='{FechaIni}' and fechaoperacion <='{FechaFin}'
"""

# Ejecutar la consulta
df_DemIni = client.query(query).to_dataframe()

c:\Users\eramirez\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [5]:
# Función para asignar los días de la semana a cada fecha, si es festivo se trata como un domingo
import holidays
co_holidays = holidays.Colombia()

def typedays(row,tipo):

     if tipo=='WeekDay':
          return row['fecha'].weekday()
     
     elif tipo=='WeekMonth':
          return (row['fecha'].day - 1) // 7 + 1
     
     elif tipo=='DayType':
          if row['fecha'] in co_holidays:
               return 3
          elif row['fecha'].weekday()==5:
               return 2
          elif row['fecha'].weekday()==6:
               return 3
          else:
               return 1
          

In [6]:
data=df_DemIni.copy()
data = data[~data['nombre'].isin(['Total','ECUADOR138','ECUADOR220','COROZO','CUATRIC','SubArea Venezuela_Corozo','SubArea Venezuela_Cuatricentenario',
                                  'SubArea Ecuador138', 'SubArea Ecuador230'])]
data['nombre'] = data['nombre'].apply(lambda x: x if x in ['ATLANTIC', 'BOLIVAR', 'GCM','CERROMAT', 'CORDOSUC', 'SubArea Atlantico','SubArea GCM',
                                                            'SubArea Cerromatoso','SubArea Bolivar','SubArea Cordoba_Sucre'] else 'Interior')

data['nombre'] = data['nombre'].replace({'ATLANTIC': 'SubArea Atlantico', 'BOLIVAR': 'SubArea Bolivar', 'GCM': 'SubArea GCM',
                                         'CERROMAT': 'SubArea Cerromatoso', 'CORDOSUC': 'SubArea Cordoba_Sucre'})

data['nombre'] = data['nombre'].replace({'SubArea Atlantico': 'SubAtlantico', 'SubArea Bolivar': 'SubBolivar', 'SubArea GCM': 'SubGCM',
                                         'SubArea Cerromatoso': 'SubCerromatoso', 'SubArea Cordoba_Sucre': 'SubCordoba-Sucre'})

for i in range(1,25):
    data = data.rename(columns={f'hora{i}': i})

data = data.melt(id_vars=['fechaoperacion', 'nombre'], 
                       value_vars=[i for i in range(1, 25)], 
                       var_name='periodo', 
                       value_name='demand')

data=data.rename(columns={'nombre':'subarea','fechaoperacion':'fecha'})
data=data.groupby(['fecha','subarea','periodo'])[['demand']].sum().round(2).reset_index()
data=data.sort_values(by=['fecha','periodo','subarea'])

# Definir tipo de día
data['day_osf']=data.apply(lambda row: typedays(row,tipo='DayType'),axis=1)
# Definir día de la semana
data['day_w']=data.apply(lambda row: typedays(row,tipo='WeekDay'),axis=1)

data['mes']=pd.to_datetime(data['fecha']).dt.month
data['year']=pd.to_datetime(data['fecha']).dt.year

data.head(7)


,fecha,subarea,periodo,demand,day_osf,day_w,mes,year
0,2024-01-01,Interior,1,4903.62,3,0,1,2024
24,2024-01-01,SubAtlantico,1,568.75,3,0,1,2024
48,2024-01-01,SubBolivar,1,563.39,3,0,1,2024
72,2024-01-01,SubCerromatoso,1,228.61,3,0,1,2024
96,2024-01-01,SubCordoba-Sucre,1,532.93,3,0,1,2024
120,2024-01-01,SubGCM,1,542.02,3,0,1,2024
1,2024-01-01,Interior,2,4691.94,3,0,1,2024


In [7]:
# Meses a comparar
l_mes=[9,10,11]
df_2024=data.copy()
df_2024=df_2024[(df_2024.year==2024) & (df_2024['mes'].isin(l_mes))]
df_2024=df_2024.groupby(['subarea','periodo','day_osf','day_w','mes'])[['demand']].mean()
df_2024=df_2024.reset_index()
df_2024=df_2024.rename(columns={'demand':'dem2024'})


df_2025=data.copy()
df_2025=df_2025[(df_2025.year==2025) & (df_2025['mes'].isin(l_mes))]
df_2025=df_2025.groupby(['subarea','periodo','day_osf','day_w','mes'])[['demand']].mean()
df_2025=df_2025.reset_index()
df_2025=df_2025.rename(columns={'demand':'dem2025'})

# Realizar merge con las llaves requeridas
df_merge=df_2024.merge(df_2025,left_on=['subarea','periodo','day_osf','day_w','mes'],right_on=['subarea','periodo','day_osf','day_w','mes'], how='inner')

df_merge['Cambio']=100*(df_merge['dem2025']-df_merge['dem2024'])/df_merge['dem2024']

df_cambio=df_merge.groupby(['subarea','periodo','day_osf','day_w'])[['Cambio']].mean()
df_cambio=df_cambio.reset_index()

# Calcular el promedio del valor absoluto por periodo
promedio_abs = df_cambio.groupby(['subarea','periodo'])['Cambio'].apply(lambda x: x.abs().mean()).rename('promedio_abs')
promedio_abs=promedio_abs.reset_index()

# Unir el promedio al dataframe y reemplazar Cambio por +promedio o -promedio según el signo original
df_cambio = df_cambio.merge(promedio_abs, left_on=['subarea','periodo'], right_on=['subarea','periodo'], how='left')
df_cambio['CambioMod'] = df_cambio['Cambio'].clip(lower=-df_cambio['promedio_abs'], upper=df_cambio['promedio_abs'])


# df_cambio.to_csv('CambioDem.csv')
df_cambio

,subarea,periodo,day_osf,day_w,Cambio,promedio_abs,CambioMod
0,Interior,1,1,0,1.828457,2.687834,1.828457
1,Interior,1,1,1,2.854207,2.687834,2.687834
2,Interior,1,1,2,2.853451,2.687834,2.687834
3,Interior,1,1,3,3.196441,2.687834,2.687834
4,Interior,1,1,4,2.868733,2.687834,2.687834
...,...,...,...,...,...,...,...
1147,SubGCM,24,1,3,14.835332,17.447846,14.835332
1148,SubGCM,24,1,4,14.637710,17.447846,14.637710
1149,SubGCM,24,2,5,17.459638,17.447846,17.447846
1150,SubGCM,24,3,0,20.732277,17.447846,17.447846


In [8]:
from datetime import date

fecha_ini_new = pd.to_datetime('2025-12-20')

year=fecha_ini_new.year
mes=fecha_ini_new.month
week=fecha_ini_new.isocalendar().week 
dia=fecha_ini_new.day
diaw=fecha_ini_new.isocalendar().weekday


fecha_new = pd.to_datetime(date(year-1, mes, dia))

weekold=fecha_new.isocalendar().week 

fecha_ini = date.fromisocalendar(year-1, weekold, diaw)
semanas=4
fecha_fin = fecha_ini + dt.timedelta(days=7*semanas-1)

# fecha=fecha + dt.timedelta(days=364)

In [9]:
fecha_ini

datetime.date(2024, 12, 21)

In [10]:
# fecha_ini = pd.to_datetime('2024-12-21')
# fecha_fin = pd.to_datetime('2025-01-17')
df_Dem=data[(data.fecha>=fecha_ini) & (data.fecha <= fecha_fin)]
df_Dem = df_Dem.sort_values(by=['fecha','subarea','periodo'], ascending=[True,True,True])
l_col=list(df_Dem.columns)

df_Dem=df_Dem.merge(df_cambio,left_on=['subarea','periodo','day_osf','day_w'],right_on=['subarea','periodo','day_osf','day_w'], how='left')[l_col + ['CambioMod']]
l_col=list(df_Dem.columns)

df_imputar = df_Dem.groupby(['subarea', 'periodo','day_osf'])['CambioMod'].mean()
df_imputar=df_imputar.reset_index()
df_imputar=df_imputar.rename(columns={'CambioMod':'CambioModAgre'})

df_Dem=df_Dem.merge(df_imputar,left_on=['subarea', 'periodo','day_osf'],right_on=['subarea', 'periodo','day_osf'], how='left')[l_col + ['CambioModAgre']]

df_Dem['CambioMod'] = df_Dem['CambioMod'].fillna(df_Dem['CambioModAgre'])

df_Dem=df_Dem[l_col]

# Verificar si hay valores faltantes en 'CambioMod'
n_missing = df_Dem['CambioMod'].isna().sum()
print("Hay valores faltantes en 'CambioMod'?:", n_missing > 0)
print("Cantidad de valores faltantes en 'CambioMod':", n_missing)
if n_missing > 0:
    print("Ejemplos de filas con missing en 'CambioMod':")
    display(df_Dem[df_Dem['CambioMod'].isna()].head())
df_Dem
df_DemIni=df_Dem.copy()
# df_Dem=df_Dem[['Fecha','Subarea','Periodo','DemMW']]

Hay valores faltantes en 'CambioMod'?: False
Cantidad de valores faltantes en 'CambioMod': 0


In [11]:
df_Dem=df_DemIni.copy()
# Fechas que se quiere con factores diferenciados
l_fechas_esp=['12-24','12-25','12-26','12-31','01-01','01-02']
factor_dnormal=1
factor_desp=0.3

# crear columna con mes-día y aplicar factores especiales o normales
df_Dem['mm_dd'] = pd.to_datetime(df_Dem['fecha']).dt.strftime('%m-%d')
df_Dem['factor_extra'] = df_Dem['mm_dd'].isin(l_fechas_esp).map({True: factor_desp, False: factor_dnormal})

# calcular demanda modificada: aplica CambioMod primero y luego el factor extra
df_Dem['demandMod'] = (df_Dem['demand'] * (1 + (df_Dem['CambioMod']*df_Dem['factor_extra'])/ 100)).round(2)

# # limpiar columnas auxiliares si no se requieren
# df_Dem.drop(columns=['mm_dd', 'factor_extra'], inplace=True)

df_Dem

,fecha,subarea,periodo,demand,day_osf,day_w,mes,year,CambioMod,mm_dd,factor_extra,demandMod
0,2024-12-21,Interior,1,5774.29,2,5,12,2024,1.862891,12-21,1.0,5881.86
1,2024-12-21,Interior,2,5531.91,2,5,12,2024,2.085522,12-21,1.0,5647.28
2,2024-12-21,Interior,3,5380.59,2,5,12,2024,2.293167,12-21,1.0,5503.98
3,2024-12-21,Interior,4,5311.28,2,5,12,2024,2.381407,12-21,1.0,5437.76
4,2024-12-21,Interior,5,5369.24,2,5,12,2024,2.353713,12-21,1.0,5495.62
...,...,...,...,...,...,...,...,...,...,...,...,...
4027,2025-01-17,SubGCM,20,846.73,1,4,1,2025,13.013647,01-17,1.0,956.92
4028,2025-01-17,SubGCM,21,867.72,1,4,1,2025,13.247842,01-17,1.0,982.67
4029,2025-01-17,SubGCM,22,828.30,1,4,1,2025,14.958271,01-17,1.0,952.20
4030,2025-01-17,SubGCM,23,828.42,1,4,1,2025,14.810408,01-17,1.0,951.11


In [12]:
# df_Dem2024Mod=df_Dem.copy()
# df_Dem2024Mod=df_Dem2024Mod[['fecha','subarea','periodo','demandMod']]
# df_Dem2024Mod.loc[df_Dem2024Mod['subarea'] == 'Interior', 'subarea'] = 'SubAntioquia'
# df_Dem2024Mod.to_csv('DemandaModificada2024.csv')

In [13]:
# import pandas as pd

# # Lista de fechas especiales (mm-dd)
# l_fechas_esp = ['12-23','12-24','12-25','12-26','12-30','12-31','01-01','01-02']

# # Copia y selección de columnas
# df_DemFinal = df_Dem.copy()
# df_DemFinal = df_DemFinal[['fecha', 'subarea', 'periodo', 'demandMod']]

# # Ajuste de subárea
# df_DemFinal.loc[df_DemFinal['subarea'] == 'Interior', 'subarea'] = 'SubAntioquia'

# # Asegurar que 'fecha' es datetime
# df_DemFinal['fecha'] = pd.to_datetime(df_DemFinal['fecha'])

# # mm-dd para identificar fechas especiales
# df_DemFinal['mm_dd'] = df_DemFinal['fecha'].dt.strftime('%m-%d')

# # Máscara de fechas especiales
# mask_esp = df_DemFinal['mm_dd'].isin(l_fechas_esp)

# # -------------------------------------------------
# # calcular la "fecha base"
# #   - fechas especiales: fecha + 1 día
# #   - demás fechas: se dejan igual
# # -------------------------------------------------
# df_DemFinal['fecha_esp'] = df_DemFinal['fecha']        # por defecto
# # df_DemFinal.loc[mask_esp, 'fecha_esp'] = df_DemFinal.loc[mask_esp, 'fecha'] + pd.Timedelta(days=1)

# # # calcular la fecha final resultante si se aplica la lógica (fecha_base + 364)
# # df_DemFinal['fecha_final_preview'] = df_DemFinal['fecha'] + pd.Timedelta(days=364)

# # # -------------------------------------------------
# # # Crear la columna final deseada:
# # #   - Si la fecha es especial → usar fecha_esp
# # #   - Si no es especial → usar fecha_final_preview
# # # -------------------------------------------------
# # df_DemFinal['fecha_final'] = df_DemFinal['fecha_final_preview']
# # df_DemFinal.loc[mask_esp, 'fecha_final'] = df_DemFinal.loc[mask_esp, 'fecha_esp']




# # # df_DemFinal.to_csv('DemandaModificada.csv')

# df_DemFinal


In [16]:
import pandas as pd
import numpy as np
from datetime import date, timedelta

# ==========================
# Parámetros
# ==========================
TARGET_START_YEAR = 2025  # año donde arranca el pronóstico
FECHAS_ESPECIALES = [
    '12-24', '12-25', '12-26',
    '12-31', '01-01', '01-02'
]
OFFSETS_BUSQUEDA = [7, 14, 21, 28]  # días para buscar vecinos (±)

# ==========================
# 1. Preparar base
# ==========================
df = df_Dem.copy()
df = df[['fecha', 'subarea', 'periodo', 'demandMod']].copy()

# Normalizar subárea
df.loc[df['subarea'] == 'Interior', 'subarea'] = 'SubAntioquia'

df['fecha'] = pd.to_datetime(df['fecha'])
df['mm_dd_orig'] = df['fecha'].dt.strftime('%m-%d')
df['es_especial_orig'] = df['mm_dd_orig'].isin(FECHAS_ESPECIALES)

fecha_ini_orig = df['fecha'].min()
fecha_fin_orig = df['fecha'].max()
dow_ini_orig   = fecha_ini_orig.dayofweek  # 0–6

# ==========================
# 2. Calcular offset alineando día de semana
# ==========================
base_candidate = date(TARGET_START_YEAR, fecha_ini_orig.month, fecha_ini_orig.day)
dow_candidate = base_candidate.weekday()

delta_forward = (dow_ini_orig - dow_candidate) % 7   # mover adelante [0..6]
delta_backward = delta_forward - 7                   # mover atrás   [-7..-1]
delta_days = delta_backward if abs(delta_backward) < abs(delta_forward) else delta_forward

fecha_ini_target = pd.Timestamp(base_candidate + timedelta(days=delta_days))
offset = fecha_ini_target - fecha_ini_orig
fecha_fin_target = fecha_fin_orig + offset

# ==========================
# 3. Aplicar offset base
# ==========================
df['fecha_orig'] = df['fecha']
df['fecha_new'] = df['fecha'] + offset

# ==========================
# 4. Construir calendario completo por (subarea, periodo)
#    manteniendo fecha_orig donde existe
# ==========================
grupos = []
for (sub, per), g in df.groupby(['subarea', 'periodo']):
    g = g[['fecha_new', 'fecha_orig', 'demandMod', 'es_especial_orig']].copy()
    g = g.sort_values('fecha_new')

    # Índice con la nueva fecha
    g = g.set_index('fecha_new')

    # Calendario completo en horizonte de pronóstico
    idx_full = pd.date_range(start=fecha_ini_target, end=fecha_fin_target, freq='D')
    g = g.reindex(idx_full)

    g['subarea'] = sub
    g['periodo'] = per

    # mm-dd y flags de especiales
    g['mm_dd'] = g.index.strftime('%m-%d')
    g['es_especial_fore'] = g['mm_dd'].isin(FECHAS_ESPECIALES)

    # es_especial_orig solo aplica a días que existen en el histórico -> rellenar NaN como False
    g['es_especial_orig'] = g['es_especial_orig'].fillna(False).astype(bool)

    grupos.append(g.reset_index().rename(columns={'index': 'fecha_new'}))

df_fore = pd.concat(grupos, ignore_index=True)

# ==========================
# 5. Perfil de ESPECIALES desde el histórico
#    (solo días especiales históricos)
# ==========================
df_hist_esp = df[df['es_especial_orig']].copy()

perfil_especial = (
    df_hist_esp.groupby(['subarea', 'periodo', 'mm_dd_orig'])['demandMod']
    .mean()
    .reset_index()
    .rename(columns={
        'mm_dd_orig': 'mm_dd',
        'demandMod': 'demandMod_esp_hist'
    })
)

# Merge del perfil especial al forecast por (subarea, periodo, mm_dd)
df_fore = df_fore.merge(
    perfil_especial,
    on=['subarea', 'periodo', 'mm_dd'],
    how='left'
)

# ==========================
# 6. RELLENO DE FECHAS ESPECIALES NUEVAS
#    SOLO con especiales del histórico
# ==========================
mask_esp_fore = df_fore['es_especial_fore']

# Si hay valor especial histórico, úsalo; si no, deja lo que haya (incluido NaN)
df_fore.loc[mask_esp_fore & df_fore['demandMod_esp_hist'].notna(), 'demandMod'] = \
    df_fore.loc[mask_esp_fore & df_fore['demandMod_esp_hist'].notna(), 'demandMod_esp_hist']

# Nota: aquí estamos cumpliendo:
# - Las fechas especiales NUEVAS toman valores desde perfiles de fechas especiales HISTÓRICAS.
# - No usamos días normales históricos para rellenar especiales.

# ==========================
# 7. CORREGIR FECHAS QUE ERAN ESPECIALES ORIGEN
#    Y AHORA SON NORMALES EN EL PRONÓSTICO
#    usando ±7, ±14, ±21, ±28 (solo desde días normales)
# ==========================
def corregir_especiales_desplazadas(gr):
    gr = gr.set_index('fecha_new').sort_index()
    vals = gr['demandMod'].copy()
    esp_fore = gr['es_especial_fore']
    esp_origen = gr['es_especial_orig']

    # Antes eran especiales, ahora NO lo son
    objetivos = gr.index[(esp_origen == True) & (esp_fore == False)]

    for f in objetivos:
        for d in OFFSETS_BUSQUEDA:
            asignado = False
            for sgn in (+1, -1):
                f_cand = f + timedelta(days=sgn * d)
                if f_cand not in vals.index:
                    continue
                # fuente debe ser NORMAL en el forecast
                if esp_fore.get(f_cand, False):
                    continue
                if pd.notna(vals.loc[f_cand]):
                    vals.loc[f] = vals.loc[f_cand]
                    asignado = True
                    break
            if asignado:
                break
        # si no encuentra nada, deja el valor como está (puede venir del offset o ser NaN)

    gr['demandMod'] = vals
    return gr.reset_index()

df_corr = (
    df_fore
    .groupby(['subarea', 'periodo'], group_keys=False)
    .apply(corregir_especiales_desplazadas)
)

# ==========================
# 8. df_final con fecha original y fecha nueva
# ==========================
df_final = df_corr[[
    'fecha_orig',     # fecha histórica (NaT si es día generado solo en forecast)
    'fecha_new',      # fecha en año de pronóstico
    'subarea',
    'periodo',
    'demandMod',
    'es_especial_fore',
    'es_especial_orig'
]].rename(columns={'es_especial_fore': 'es_especial'})

df_final = df_final.sort_values(['fecha_orig','subarea', 'periodo']).reset_index(drop=True)

df_f=df_final[['fecha_new','subarea','periodo','demandMod']]
df_f.to_csv('DemandaModificada.csv')

C:\Users\eramirez\AppData\Local\Temp\ipykernel_13296\3283159754.py:157: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(corregir_especiales_desplazadas)


In [17]:
df_final

,fecha_orig,fecha_new,subarea,periodo,demandMod,es_especial,es_especial_orig
0,2024-12-21,2025-12-20,SubAntioquia,1,5881.86,False,False
1,2024-12-21,2025-12-20,SubAntioquia,2,5647.28,False,False
2,2024-12-21,2025-12-20,SubAntioquia,3,5503.98,False,False
3,2024-12-21,2025-12-20,SubAntioquia,4,5437.76,False,False
4,2024-12-21,2025-12-20,SubAntioquia,5,5495.62,False,False
...,...,...,...,...,...,...,...
4027,2025-01-17,2026-01-16,SubGCM,20,956.92,False,False
4028,2025-01-17,2026-01-16,SubGCM,21,982.67,False,False
4029,2025-01-17,2026-01-16,SubGCM,22,952.20,False,False
4030,2025-01-17,2026-01-16,SubGCM,23,951.11,False,False
